In [4]:
# 🧠 UNILAG Admission Inference Notebook

import pandas as pd
import joblib
import numpy as np

model_path = "../../Models/unilag_admission_model.pkl"
model = joblib.load(model_path)
print("Model loaded successfully!")

# Example 1️⃣ – Predict for a single applicant
# Sample applicant data
single_applicant = pd.DataFrame([{
    "Faculty": "Engineering",
    "Department": "Computer Engineering",
    "UTME_Score": 290,
    "Post_UTME_Score": 20,   # Post-UTME score (max 30)
    "Olevel_Valid": 1,       # 1 if all grades ≥ C6
    "olevel_avg_points": 3.8 # average from 5 subjects
}])

# Predict admission outcome
single_prediction = model.predict(single_applicant)[0]
label_map = {0: "Not Admitted", 1: "Considered", 2: "Admitted"}
print("\n🎓 Single Applicant Prediction:")
print(f"Prediction: {label_map[single_prediction]}")

# Example 2️⃣ – Predict for multiple applicants

multiple_applicants = pd.DataFrame([
    {
        "Faculty": "Engineering",
        "Department": "Electrical & Electronics Engineering",
        "UTME_Score": 310,
        "Post_UTME_Score": 23,
        "Olevel_Valid": 1,
        "olevel_avg_points": 3.6
    },
    {
        "Faculty": "Management Sciences",
        "Department": "Accounting",
        "UTME_Score": 260,
        "Post_UTME_Score": 15,
        "Olevel_Valid": 1,
        "olevel_avg_points": 3.0
    },
    {
        "Faculty": "Arts",
        "Department": "English",
        "UTME_Score": 245,
        "Post_UTME_Score": 17,
        "Olevel_Valid": 1,
        "olevel_avg_points": 3.4
    }
])

# Predict for multiple applicants
multi_predictions = model.predict(multiple_applicants)
multiple_applicants["Predicted_Status"] = [label_map[p] for p in multi_predictions]

print("\n📊 Multiple Applicants Prediction Results:")
print(multiple_applicants)


Model loaded successfully!


ValueError: columns are missing: {'Aggregate'}

In [1]:
import pandas as pd
import joblib

model = joblib.load("../../Models/unilag_admission_model.pkl")
print("Model loaded successfully!")


Model loaded successfully!


In [18]:
olevel_scale = {'A1': 4.0, 'B2': 3.6, 'B3': 3.2, 'C4': 2.8, 'C5': 2.4, 'C6': 2.0}

def calculate_olevel_avg(grades):
    """Compute average O'Level grade points (UNILAG scale)."""
    points = [olevel_scale[g] for g in grades]
    return round(sum(points) / len(points), 2)

def calculate_aggregate(utme, post_utme, olevel_avg):
    """Compute UNILAG weighted aggregate score: 50% UTME + 30% Post-UTME + 20% O'Level."""
    utme_score = (utme / 400) * 50
    post_utme_score = (post_utme / 30) * 30
    olevel_points = olevel_avg * 5  # scaled to 20%
    total = utme_score + post_utme_score + olevel_points
    return round(total, 2)

# Merit cut-off marks (2021/2022 sample values from UNILAG website – extend with full PDF data)
unilag_cutoffs = {
    "Medicine and Surgery": 80.8,
    "Dentistry": 77.4,
    "Nursing": 70.875,
    "Pharmacy": 76.125,
    "Medical Laboratory Science": 74.675,
    "Pharmacology": 72.625,
    "Physiology": 71.55,
    "Physiotherapy": 73.75,
    "Radiography": 72.525,
    "Law": 68.65,
    "Accounting": 74.1,
    "Business Administration": 68.65,
    "Actuarial Science": 61.875,
    "Finance": 68.9,
    "Insurance": 67.125,
    "ER & HRM": 66.95,
    "Computer Science": 71.75,
    "Mass Communication": 72.35,
    "Economics": 69.95,
    "Political Science": 68.6,
    "Psychology": 67.4,
    "Sociology": 66.15,
    "Creative Arts": 67.05,
    "English": 68.175,
    "French": 66.55,
    "Russian": 67.65,
    "History & Strategic Studies": 68.6,
    "Linguistic Igbo/Yoruba": 61.175,
    "Chinese": 66.7,
    "Philosophy": 67.4,
    "Christian Religious Studies": 61.625,
    "Islamic Religious Studies": 60.225,
    # Default fallback
    "Other": 50
}

# -----------------------------
# 3️⃣ Applicant Data (Modify This Section)
# -----------------------------
applicant = {
    "Faculty": "Social Sciences",
    "Department": "Economics",
    "UTME_Score": 265,
    "Post_UTME_Score": 24.5,  # out of 30
    "Olevel_Grades": ["B3", "B2", "C4", "A1", "C5"],
    "Olevel_Valid": 1,  # 1 = passed, 0 = failed
}

# -----------------------------
# 4️⃣ Process Input
# -----------------------------
# Compute average O'Level points
olevel_avg_points = calculate_olevel_avg(applicant["Olevel_Grades"])

# Auto-fetch department cutoff
applicant_cutoff = unilag_cutoffs.get(applicant["Department"], 50)

# Compute aggregate score
aggregate = calculate_aggregate(
    applicant["UTME_Score"],
    applicant["Post_UTME_Score"],
    olevel_avg_points
)

# Assemble applicant DataFrame
input_df = pd.DataFrame([{
    "Faculty": applicant["Faculty"],
    "Department": applicant["Department"],
    "UTME_Score": applicant["UTME_Score"],
    "Post_UTME_Score": applicant["Post_UTME_Score"],
    "Aggregate": aggregate,
    "Cutoff": applicant_cutoff,
    "Olevel_Valid": applicant["Olevel_Valid"],
    "olevel_avg_points": olevel_avg_points
}])

print("\n📋 Applicant Data:")
print(input_df)

# -----------------------------
# 5️⃣ Predict Admission Probability
# -----------------------------
probs = model.predict_proba(input_df)[0]
probability = probs[2] # Probability of being admitted
chance_percent = probability * 100

# Threshold-based decision
if probability >= 0.7:
    chance_label = "High"
elif 0.5 <= probability < 0.7:
    chance_label = "Moderate"
else:
    chance_label = "Low"

# -----------------------------
# 6️⃣ Display Results
# -----------------------------
print("\n🎯 Prediction Result:")
print(f"📊 Aggregate: {aggregate}")
print(f"📘 O'Level Average Points: {olevel_avg_points}")
print(f"💡 Admission Probability: {chance_percent:.2f}% ({chance_label} Chance)")

if chance_label == "High":
    print("✅ Likely to be admitted into UNILAG!")
elif chance_label == "Moderate":
    print("⚖️ Fair chance — performance is competitive, but not guaranteed.")
else:
    print("❌ Low chance of admission — consider changing course.")



📋 Applicant Data:
           Faculty Department  UTME_Score  Post_UTME_Score  Aggregate  Cutoff  \
0  Social Sciences  Economics         265             24.5      73.62   69.95   

   Olevel_Valid  olevel_avg_points  
0             1                3.2  

🎯 Prediction Result:
📊 Aggregate: 73.62
📘 O'Level Average Points: 3.2
💡 Admission Probability: 78.51% (High Chance)
✅ Likely to be admitted into UNILAG!
